In [1]:
import os
import io
from copy import deepcopy
from pathlib import Path
from PIL import Image as PILImage
import cv2
import numpy as np
import re

from IPython.display import display, Audio
from ipywidgets import widgets, interact

from utils.photo_utils import exif_rotate, get_photo, get_centermost_detections
from cloud_ocr.google_api import save_cuts_for_detection, detect_text, draw_ocr
import cloud_ocr.ocr_utils as ocr_fcns

home_dir = os.getcwd()
print("Welcome!")

Welcome!


In [2]:
# TODO: make it simple, this is workaround
os.chdir(Path(home_dir, "yolov5"))
from model import Model
os.chdir(home_dir)

In [3]:
# pricetag model
yolo_price_weight_path = str(Path("yolov5", "weights", "pricetag_small.pt"))
yolo_price_model = Model(yolo_price_weight_path, device="cpu")  # ~ 0.2s on CPU 

# product name and price model
yolo_price_weight_path = str(Path("yolov5", "weights", "product_name_and_price_small_albert_320.pt"))
yolo_product_name_model = Model(yolo_price_weight_path, device="cpu")  # ~ 0.2s on CPU 
yolo_product_name_model.img_size = 320

print("Models loaded")

Models loaded


In [23]:
def callback(value):
    if btn_upload.data:

        # =============================================================================
        # INITIALIZE IMAGE
        # =============================================================================

        print("You upload a file:")
        im_file = get_photo(btn_upload)

        # =============================================================================
        # DETECT PRICETAGS
        # =============================================================================
        try: 
            det_pricetags = yolo_price_model.detect([np.array(im_file)],  mode="return")

            # =============================================================================
            # CUT PRICETAGS
            # =============================================================================        

            # returns list of n centermost pricetags. To return only coordinates, use mode="return" 
            print(f"{len(det_pricetags['cls'])} pricetags found, working with centermost pricetag")

            # TODO: filter pricetag and read it
            # returns list of n centermost pricetags. To return only coordinates, use mode="return" 
            cuts = get_centermost_detections(im_file,
                                                      det_pricetags,
                                                      n=1,
                                                      mode="cut",
                                                      bigger_box_by_pct=0.01
                                                      )

            display(PILImage.fromarray(cuts[0]))

            # =============================================================================
            # READ PRICETAGS
            # =============================================================================   

            # saving cuts to be able to run Google API detection (custom load)
            cuts_path = save_cuts_for_detection(cuts)

            # run Google API text detection and OCR
            cuts_ocr = [detect_text(path) for path in cuts_path]

            # =============================================================================
            # CLASSIFY DETECTIONS
            # =============================================================================

            product_name = cuts_ocr[0][1]
            x = cuts_ocr[1][1]

            # heuristic 1
            if len(x) == 4:
                # heuristic 2
                if re.match(heur_2_string, x[1]) and not re.match(heur_2_string, x[2]):
                    swap = x[1]
                    x[1] = x[2]
                    x[2] = swap

            # heuristic 2
            x[-1] = re.sub(r"K.$","Kč",x[-1])
            if re.match(r"Kč",x[-1]):
                del x[-1]

            # heuristic 3
            if len(x) < 4:
                # split off haller
                if re.match(heur_3_string, x[-1]):
                    x.append(x[-1][-2:])
                    x[-2] = x[-2][:-2]

            #heuristic 5
            for i in range(1,len(x)):
                x[i] = re.sub(r"\D","",x[i])

            price = x 
            # print results 
            print(product_name[0])  
            print(",".join(price[1:]))
        except:
            print("Nedokážu přečíst obrázek, zkuste to prosím znovu.")

    return

In [24]:
btn_upload = widgets.FileUpload()

In [25]:
_ = interact(callback, value=btn_upload)

interactive(children=(FileUpload(value={}, description='Upload'), Output()), _dom_classes=('widget-interact',)…